In [1]:
import itertools
from tqdm import tqdm

In [2]:
filename = "sample.txt"
# filename = "input.txt"
with open(filename, encoding="utf-8") as f:
    data = f.read()

lines = data.strip().split("\n")

### Improvements from reddit:
- Simplified rotation to direction *= 1j
- Replaced deepcopy of obstacle dict in part 2 with dict merge (which copies). Speeds up final loop from ~80it/s to ~400it/s

In [3]:
# Parse the map
obstacles = dict()
for y, line in enumerate(lines):
    for x, c in enumerate(line):
        pos = x + y * 1j
        obstacles[pos] = c == "#"
        if c == "^":
            starting_pos = pos

In [4]:
def walk(obstacles: dict, starting_pos: complex) -> tuple[bool, set[tuple[complex, int]]]:
    guard_pos = starting_pos
    direction = -1j
    seen = {(guard_pos, direction)}
    for steps in itertools.count(1):
        # Try to take a step
        next_pos = guard_pos + direction
        try:
            is_blocked = obstacles[next_pos]
        except KeyError:
            # print(f"Next pos {next_pos} is out of bounds. Done!")
            return True, seen
        
        # Loop detection: Have I been here before?
        if (next_pos, direction) in seen:
            # print(f"Loop found, reached {guard_pos} again in {steps} steps!")
            return False, seen
    
        if is_blocked:
            # Turn clockwise and repeat
            direction *= 1j
        else:
            guard_pos = next_pos
        # Add new (pos, direction) regardless if the change was from turning or stepping
        seen.add((guard_pos, direction))

In [5]:
## Part 1
# The guard ^ starts facing N and moves forwards, turning clockwise when it hits an obstacle
# Including starting pos, how many distinct positions are visited before it leaves the map?

guard_escapes, p1_seen = walk(obstacles, starting_pos)
p1_route = set(pos for pos, _ in p1_seen)
len(p1_route)

41

In [6]:
## Part 2
# By adding a single obstacle to any open pos (except the starting guard pos), create a loop that traps the guard
# How many different positions could you choose for the obstruction?

# A loop exists if the guard reaches the same position facing the same direction
# It _might_ be ok for the obstruction to be 1 out-of-bounds (but let's ignore that for attempt 1)

# Brute-force: We can iteratively try all open spots in obstacles. This would be max 130x130 attempts
# Slightly better: Try all cells in the initial route
#  We don't need to consider cells outside this since we're only placing 1 obstacle

In [7]:
# Try all cells in the initial route, keep track of how many caused a loop
valid_candidate_count = 0
# Can't put an obstacle on starting pos
initial_route = p1_route.difference({starting_pos})

for obstacle_candidate in tqdm(initial_route, desc="Obstacle position candidates"):
    new_obstacles = obstacles | {obstacle_candidate: True}
    guard_escapes, path = walk(new_obstacles, starting_pos)
    if not guard_escapes:
        valid_candidate_count += 1

valid_candidate_count

Obstacle position candidates: 100%|██████████| 40/40 [00:00<00:00, 36743.79it/s]


6